In [1]:
import mlinsights.mlmodel
import sklearn
import pandas as pd
import numpy as np
import pickle
from sklearn.manifold import TSNE
from mlinsights.mlmodel import PredictableTSNE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import tensorflow_hub as hub
# Tokenizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import sent_tokenize, word_tokenize

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

data_T=pd.read_csv('C://Users//antoi//Dropbox//PC//Desktop//Projet 5//data_T.csv')
features_USE=np.loadtxt('features_USE.csv', delimiter=",")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


FileNotFoundError: features_USE.csv not found.

In [7]:
def tokenizer_fct(sentence) :
    # print(sentence)
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ').replace('<', ' ').replace('>', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

# Stop words
from nltk.corpus import stopwords
stop_w = list(set(stopwords.words('english'))) + ['[', ']', ',', '.', ':', '?', '(', ')']

def stop_word_filter_fct(list_words) :
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

# lower case et alpha
def lower_start_fct(list_words) :
    lw = [w.lower() for w in list_words if (not w.startswith("@"))
                                       and (not w.startswith("http"))]
    return lw

def transform_dl_fct(desc_text) :
    word_tokens = tokenizer_fct(desc_text)
    lw = lower_start_fct(word_tokens)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

def feature_USE_fct(sentences, b_size) :
    batch_size = b_size
    features=np.array([])
    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        feat = embed(sentences[idx:idx+batch_size])
        if step==0 :
            features = feat
        else :
            features = np.concatenate((features,feat))
    return features


def Jaccard_fct_Forest(features) :
    ptsne = mlinsights.mlmodel.PredictableTSNE()
    X_tsne = ptsne.fit_transform(features,data_T['tags'].values)
    minlen=min(X_tsne.shape[0],data_T['tags'].shape[0])
    x_train,x_test,y_train,y_test=train_test_split(X_tsne[:minlen],data_T['tags'].values[:minlen],test_size=0.1)
    # Détermination des clusters à partir des données après Tsne
    cls = sklearn.ensemble.RandomForestClassifier()
    cls.fit(x_train,y_train)

    return cls, ptsne

In [8]:
model_USE, ptsneUSE = Jaccard_fct_Forest(features_USE)

c:\users\antoi\pycharmprojects\projet 5\venv\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\users\antoi\pycharmprojects\projet 5\venv\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\users\antoi\pycharmprojects\projet 5\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


25.4


* En se basant sur la metric utilisée, le modèle RandomForest est le plus performant.
* En terme de traitements de données, le modèle USE est le meilleur.

On exporte alors les modèles nécessaires pour l'entrée d'API.

In [9]:
pickle.dump(model_USE,open('USE.pkl','wb'))
USE=pickle.load(open('USE.pkl','rb'))

pickle.dump(ptsneUSE,open('tsne.pkl','wb'))
tsne=pickle.load(open('tsne.pkl','rb'))
